In [1]:
from clickhouse_driver import Client
from pandas import DataFrame

CLICKHOUSE_HOST = 'localhost'
COMPRESSION = 'lz4'
client = Client(host=CLICKHOUSE_HOST, compression=True)
execute = client.execute

def load_from_ch_to_df(sql_query, columns=None):
    return DataFrame(execute(sql_query), columns=columns)

##### Простейшие характеристики графов 
<ul>
<li> Средняя степень графа (валентность) - подсчет кол-ва ребер на вершинах, распределение
<li> концевые вершины (только одна связь)
<li> изолированные триплеты (subject и object в единственном числе)
<li> доминирующие вершины - максимальное кол-во связей
<li> симметричность - % субъектов, которые являются объектами (и наоборот) </ul>

In [2]:
#Валентность
execute("USE kg;")

df = load_from_ch_to_df('''SELECT source, 
                            count(distinct subject) as subjects,
                            count(distinct object)  as objects,
                            count(*) as edges
                            FROM caligraph
                            group by source''', columns=['source', 'subjects', 'objects', 'edges'])
df

,source,subjects,objects,edges
0,dbpedia_caligraph-relations.nt,3753336,69761,8752268
1,caligraph-metadata.nt,1,9,11
2,dbpedia_caligraph-instances.nt,6136115,6,6293575
3,caligraph-ontology_dbpedia-mapping.nt,1157,1155,1184
4,caligraph-instances_types.nt,14827038,877672,56599829
5,caligraph-instances_relations.nt,4557400,147387,12989198
6,caligraph-instances_transitive-types.nt,13370627,287292,186784124
7,caligraph-ontology.nt,1189839,519342,3396812
8,caligraph-instances_dbpedia-mapping.nt,8694651,8694651,8696730
9,caligraph-ontology_provenance.nt,1061434,1151950,1154447


In [50]:
df['subj-to-edges'] = df.apply(lambda x: round(x['subjects']/x['edges'], 3), axis=1)
df['obj-to-edges'] = df.apply(lambda x: round(x['objects']/x['edges'], 3), axis=1)

df['intensity']  = df.apply(lambda x: 2*x['edges']/(x['subjects']*(x['objects']-1)), axis=1)


df.style.background_gradient(subset=['subj-to-edges', 'obj-to-edges'], cmap='Spectral')

,source,subjects,objects,edges,subj-to-edges,obj-to-edges,intensity
0,dbpedia_caligraph-relations.nt,3753336,69761,8752268,0.429,0.008,6.68539e-05
1,caligraph-metadata.nt,1,9,11,0.091,0.818,2.75
2,dbpedia_caligraph-instances.nt,6136115,6,6293575,0.975,0,0.410264
3,caligraph-ontology_dbpedia-mapping.nt,1157,1155,1184,0.977,0.976,0.00177355
4,caligraph-instances_types.nt,14827038,877672,56599829,0.262,0.016,8.69879e-06
5,caligraph-instances_relations.nt,4557400,147387,12989198,0.351,0.011,3.86758e-05
6,caligraph-instances_transitive-types.nt,13370627,287292,186784124,0.072,0.002,9.72515e-05
7,caligraph-ontology.nt,1189839,519342,3396812,0.35,0.153,1.09941e-05
8,caligraph-instances_dbpedia-mapping.nt,8694651,8694651,8696730,1,1,2.30082e-07
9,caligraph-ontology_provenance.nt,1061434,1151950,1154447,0.919,0.998,1.88833e-06


все графы сильно разрежены <br>

<i>caligraph-ontology.nt</i> <br>граф с хорошей валетностью и балансом между объектами и субъектами для дальнейшего исследования


##### Коэффциент кластеризации графа
<ul>
    <li> кол-во замкнутых треугольников
    <li> ср. от локальных коэф кластеризац - кол-во связей в окрестности по отношению к возможному
    <li> кластеры (наборы вершин + окрестностей) с самым высоким коэф
</ul>

###### Расстояния в графе
<ul><li> Диаметры, эксцентриситеты, радиусы
<li> Мосты и шарниры
<li> Модульность - высокая плотность внутри, слабая снаружи </ul>